In [1]:
import cv2
import numpy as np
import glob
import os
from pprint import pprint
from typing import Tuple, Dict

# --- preprocessing
def preprocess(
    img: np.ndarray, 
    size: Tuple[int, int] = (600, 1200)
) -> np.ndarray:
    """
    Preprocess an image by converting it to grayscale, resizing, and applying Gaussian blur.

    Args:
        img (np.ndarray): Input image (BGR or grayscale).
        size (Tuple[int, int], optional): Target size (height, width). Defaults to (600, 1200).

    Returns:
        np.ndarray: Preprocessed grayscale image.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    h, w = size
    gray = cv2.resize(gray, (w, h))
    gray = cv2.GaussianBlur(gray, (3,3), 0)
    
    return gray

# --- template match (global)
def template_score(
    test: np.ndarray, 
    template: np.ndarray
) -> float:
    """
    Compute similarity score between two images using template matching.

    Args:
        test (np.ndarray): Test image.
        template (np.ndarray): Template image.

    Returns:
        float: Maximum normalized correlation coefficient (0.0–1.0).
    """
    res = cv2.matchTemplate(
        image=test, 
        templ=template, 
        method=cv2.TM_CCOEFF_NORMED
    )
    return float(np.max(res))

# --- feature match (ORB)
def orb_match_score(
    img1: np.ndarray, 
    img2: np.ndarray
) -> float:
    """
    Compute similarity score between two images using ORB feature matching.

    Args:
        img1 (np.ndarray): First image (grayscale or BGR).
        img2 (np.ndarray): Second image (grayscale or BGR).

    Returns:
        float: Feature matching score (0.0–1.0).
    """
    orb = cv2.ORB_create(1000) # type: ignore
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    if des1 is None or des2 is None or len(kp1)==0 or len(kp2)==0:
        return 0.0
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    good = [m for m in matches if m.distance < 60]
    score = len(good) / max(len(kp1), len(kp2))
    
    return float(min(score, 1.0))

# --- load prototypes (one per category)
def load_prototypes(path: str) -> Dict[str, np.ndarray]:
    prototypes = {}
    for cat_dir in os.listdir(path):
        cat_path = os.path.join(path, cat_dir)
        
        if not os.path.isdir(cat_path):
            continue
        
        imgs = (
            glob.glob(os.path.join(cat_path, "*.jpg")) + 
            glob.glob(os.path.join(cat_path, "*.png"))
        )
        
        if not imgs:
            continue
        
        img = cv2.imread(imgs[0])
        
        if img is None:
            continue
        
        prototypes[cat_dir] = preprocess(img)
        
    return prototypes

# --- classify category by prototype similarity
def detect_category(
    test_img: np.ndarray, 
    prototypes: Dict[str, np.ndarray]
) -> Tuple[str, Dict[str, float]]:
    """
    Detect the most similar category for a given image based on template similarity.

    Args:
        test_img (np.ndarray): The input image to classify.
        prototypes (Dict[str, np.ndarray]): Dictionary of category prototypes.

    Returns:
        Tuple[str, Dict[str, float]]:
            - The best-matching category name.
            - A dictionary of similarity scores for all categories.
    """
    test = preprocess(test_img)
    scores: Dict[str, float] = {
        cat: template_score(test, proto) for cat, proto in prototypes.items()
    }
    best_cat = max(scores, key=lambda k: scores[k]) if scores else "unknown"
    
    return best_cat, scores

# --- full check inside category (compare test to all templates in that category)
def category_check(
    test_img: np.ndarray, 
    template_dir: str
) -> Dict[str, float]:
    """
    Perform detailed similarity check of a test image against all templates in a category.

    Args:
        test_img (np.ndarray): Test image.
        template_dir (str): Directory containing category templates.

    Returns:
        Dict[str, float]: Dictionary with 'template', 'feature', and 'final' similarity scores.
    """
    test = preprocess(test_img)
    tpaths = (
        glob.glob(os.path.join(template_dir, "*.jpg")) + 
        glob.glob(os.path.join(template_dir, "*.png"))
    )
    
    best_t_score = 0.0
    best_f_score = 0.0
    
    for p in tpaths:
        tpl = cv2.imread(p)
        
        if tpl is None:
            continue
        
        tpl = preprocess(tpl)
        
        ts = template_score(test, tpl)
        fs = orb_match_score(test, tpl)
        best_t_score = max(best_t_score, ts)
        best_f_score = max(best_f_score, fs)
    
    final = 0.45 * best_t_score + 0.5* best_f_score
    
    return {
        "template": best_t_score, 
        "feature": best_f_score, 
        "final": final
    }

In [3]:
# Load prototypes
prototypes = load_prototypes("../input/templates-receipts/")
print("Loaded prototypes:", list(prototypes.keys()))

# Load a test image
test_path = "../input/test_images/test_transfer_fake_001.png"
img = cv2.imread(test_path)

if img is None:
    print("❌ Cannot read test image. Check path.")
    exit()

# Detect category
cat, cat_scores = detect_category(img, prototypes)
print(f"\nDetected category: {cat}")
print("Category scores:")
pprint(cat_scores)

# Run deeper check
check = category_check(img, os.path.join("../input/templates-receipts/", cat))
print("\nLayout check results:")
pprint(check)

if check["final"] >= 0.75:
    print("\n✅ Layout looks genuine")
elif check["final"] >= 0.5:
    print("\n⚠️  Layout uncertain — manual review suggested")
else:
    print("\n❌ Layout suspicious or fake")


Loaded prototypes: ['common', 'qris', 'topup', 'transfer']

Detected category: qris
Category scores:
{'common': 0.040128111839294434,
 'qris': 0.16496488451957703,
 'topup': 0.032334234565496445,
 'transfer': 0.05638188123703003}

Layout check results:
{'feature': 0.343,
 'final': 0.31640962773561476,
 'template': 0.32202139496803284}

❌ Layout suspicious or fake
